<a href="https://colab.research.google.com/github/lewisnjue/kaggle_competition_house_price/blob/main/house_price_prediction_pytorch_competion_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# let me upload my kaggle api key programatically
from google.colab import files
files.upload()
# this is the programmatically way to putting your api keys to google colab use it in your code

{}

In [ ]:
!pip install kaggle
''' for many casses teh kaggle is aready installed  '''
# this will install kaggle

' for many casses teh kaggle is aready installed  '

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# i have also changed the permission of the file for me to read and write but others have no permission to it

In [ ]:
# download the zip file now
!kaggle competitions download -c house-prices-advanced-regression-techniques


house-prices-advanced-regression-techniques.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# unzip the file like this
!unzip house-prices-advanced-regression-techniques.zip


Archive:  house-prices-advanced-regression-techniques.zip
replace data_description.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [74]:
!ls

data_description.txt				 sample_data		test.csv
house-prices-advanced-regression-techniques.zip  sample_submission.csv	train.csv


In [75]:
# create a dataframe for my train data import the libraries am going to use
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
%matplotlib inline
import torch
from torch import nn

import pandas as pd
df = pd.read_csv('train.csv',index_col='Id')


In [76]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [77]:
len(list(df.columns))

80

In [78]:
(df.isna()).sum()

,0
MSSubClass,0
MSZoning,0
LotFrontage,259
LotArea,0
Street,0
...,...
MoSold,0
YrSold,0
SaleType,0
SaleCondition,0


In [79]:
df.describe()


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [80]:
''' numerical_cols = df.select_dtypes(include=np.number).columns # this will get me the numerical columns
for col in numerical_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mean())

# Categorical columns with null values dropped
categorical_cols = df.select_dtypes(exclude=np.number).columns
for col in categorical_cols:
    if df[col].isnull().any():
        df.dropna(subset=[col], inplace=True) '''

numerical_columns = df.select_dtypes(include=np.number).columns
categorical_columns = df.select_dtypes(exclude=np.number).columns
list(numerical_columns),list(categorical_columns)
df[list(numerical_columns)] = df[list(numerical_columns)].fillna(df[list(numerical_columns)].mean())
df[list(categorical_columns)] = df[list(categorical_columns)].fillna('Missing')
df.head()


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,Missing,Reg,Lvl,AllPub,Inside,...,0,Missing,Missing,Missing,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,Missing,Reg,Lvl,AllPub,FR2,...,0,Missing,Missing,Missing,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,Missing,IR1,Lvl,AllPub,Inside,...,0,Missing,Missing,Missing,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,Missing,IR1,Lvl,AllPub,Corner,...,0,Missing,Missing,Missing,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,Missing,IR1,Lvl,AllPub,FR2,...,0,Missing,Missing,Missing,0,12,2008,WD,Normal,250000


In [81]:
df.select_dtypes(include=np.number).columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

In [82]:
df.select_dtypes(exclude=np.number).columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [87]:
df['SalePrice'].isna().sum()

0

,0
MSSubClass,0
MSZoning,0
LotFrontage,0
LotArea,0
Street,0
...,...
MoSold,0
YrSold,0
SaleType,0
SaleCondition,0


In [88]:
# now in my data i have no null values -> this is very good now for my data predictions
''' let me now separate the target columns form the train columns '''
target = df['SalePrice']
labels = df.drop('SalePrice',axis=1)

In [90]:
target.shape,labels.shape

((1460,), (1460, 79))

In [106]:
''' let me normilize my data '''
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numerical_cols_no_target = list(numerical_columns)[:-1]
labels[numerical_cols_no_target] = scaler.fit_transform(labels[numerical_cols_no_target])
labels.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,0.073375,RL,-0.229372,-0.207142,Pave,Missing,Reg,Lvl,AllPub,Inside,...,-0.270208,-0.068692,Missing,Missing,Missing,-0.087688,-1.599111,0.138777,WD,Normal
2,-0.872563,RL,0.451936,-0.091886,Pave,Missing,Reg,Lvl,AllPub,FR2,...,-0.270208,-0.068692,Missing,Missing,Missing,-0.087688,-0.489110,-0.614439,WD,Normal
3,0.073375,RL,-0.093110,0.073480,Pave,Missing,IR1,Lvl,AllPub,Inside,...,-0.270208,-0.068692,Missing,Missing,Missing,-0.087688,0.990891,0.138777,WD,Normal
4,0.309859,RL,-0.456474,-0.096897,Pave,Missing,IR1,Lvl,AllPub,Corner,...,-0.270208,-0.068692,Missing,Missing,Missing,-0.087688,-1.599111,-1.367655,WD,Abnorml
5,0.073375,RL,0.633618,0.375148,Pave,Missing,IR1,Lvl,AllPub,FR2,...,-0.270208,-0.068692,Missing,Missing,Missing,-0.087688,2.100892,0.138777,WD,Normal


In [107]:
'''  do one hot encoding  for our categorical columns '''
labels = pd.get_dummies(labels,drop_first=True,dtype=int)
labels.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,0.073375,-0.229372,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.511418,0.575425,-0.288653,...,0,0,0,0,1,0,0,0,1,0
2,-0.872563,0.451936,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.574410,1.171992,-0.288653,...,0,0,0,0,1,0,0,0,1,0
3,0.073375,-0.093110,0.073480,0.651479,-0.517200,0.984752,0.830215,0.323060,0.092907,-0.288653,...,0,0,0,0,1,0,0,0,1,0
4,0.309859,-0.456474,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.574410,-0.499274,-0.288653,...,0,0,0,0,1,0,0,0,0,0
5,0.073375,0.633618,0.375148,1.374795,-0.517200,0.951632,0.733308,1.364570,0.463568,-0.288653,...,0,0,0,0,1,0,0,0,1,0


In [110]:
'''  now convert my labels to tensor excluding the id column using pytorch '''
labels = torch.from_numpy(labels.values).float()
target = torch.from_numpy(target.values).float()

In [111]:
labels

tensor([[ 0.0734, -0.2294, -0.2071,  ...,  0.0000,  1.0000,  0.0000],
        [-0.8726,  0.4519, -0.0919,  ...,  0.0000,  1.0000,  0.0000],
        [ 0.0734, -0.0931,  0.0735,  ...,  0.0000,  1.0000,  0.0000],
        ...,
        [ 0.3099, -0.1840, -0.1478,  ...,  0.0000,  1.0000,  0.0000],
        [-0.8726, -0.0931, -0.0802,  ...,  0.0000,  1.0000,  0.0000],
        [-0.8726,  0.2248, -0.0581,  ...,  0.0000,  1.0000,  0.0000]])

In [112]:
labels.shape

torch.Size([1460, 260])

In [113]:
labels.dtype

torch.float32

In [114]:
target.shape

torch.Size([1460])

In [117]:
''' create a model  '''
class regression_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_features=260, out_features=200),  # Adjusted from 331 to 260
            nn.ReLU(),
            nn.Linear(in_features=200, out_features=100),
            nn.ReLU(),
            nn.Linear(in_features=100, out_features=50),
            nn.ReLU(),
            nn.Linear(in_features=50, out_features=1))

    def forward(self, x):
        return self.layers(x)

In [118]:
''' create a loss function and an optimzer '''
model = regression_model()
loss_fun = nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.001)

In [120]:
''' do some dummy prediction '''
y_pred = model(labels.float())
y_pred

tensor([[0.1079],
        [0.1027],
        [0.1073],
        ...,
        [0.1111],
        [0.1006],
        [0.0926]], grad_fn=<AddmmBackward0>)

In [121]:
# let now train our model
epochs = 1000
for epoch in range(epochs):
    model.train()
    y_pred = model(labels.float())
    loss = loss_fun(y_pred,target.float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model.eval()
    with torch.inference_mode():
        y_pred = model(labels.float())
        test_loss = loss_fun(y_pred,target.float())
        print(f"Epoch: {epoch} | Loss: {loss} | Test Loss: {test_loss}")


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([1460])) that is different to the input size (torch.Size([1460, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0 | Loss: 39039229952.0 | Test Loss: 39039205376.0
Epoch: 1 | Loss: 39039205376.0 | Test Loss: 39039188992.0
Epoch: 2 | Loss: 39039188992.0 | Test Loss: 39039172608.0
Epoch: 3 | Loss: 39039172608.0 | Test Loss: 39039148032.0
Epoch: 4 | Loss: 39039148032.0 | Test Loss: 39039119360.0
Epoch: 5 | Loss: 39039119360.0 | Test Loss: 39039090688.0
Epoch: 6 | Loss: 39039090688.0 | Test Loss: 39039053824.0
Epoch: 7 | Loss: 39039053824.0 | Test Loss: 39039004672.0
Epoch: 8 | Loss: 39039004672.0 | Test Loss: 39038947328.0
Epoch: 9 | Loss: 39038947328.0 | Test Loss: 39038873600.0
Epoch: 10 | Loss: 39038873600.0 | Test Loss: 39038787584.0
Epoch: 11 | Loss: 39038787584.0 | Test Loss: 39038681088.0
Epoch: 12 | Loss: 39038681088.0 | Test Loss: 39038562304.0
Epoch: 13 | Loss: 39038562304.0 | Test Loss: 39038414848.0
Epoch: 14 | Loss: 39038414848.0 | Test Loss: 39038238720.0
Epoch: 15 | Loss: 39038238720.0 | Test Loss: 39038038016.0
Epoch: 16 | Loss: 39038038016.0 | Test Loss: 39037804544.0
Epoch: 

In [122]:
''' evaluate the model '''
model.eval()
with torch.inference_mode():
    y_pred = model(labels.float())
    loss = loss_fun(y_pred,target.float())
    print(f"Loss: {loss}")
''' compare the real value with the predicted one  '''
target[:5],y_pred[:5]

Loss: 6336871936.0


(tensor([208500., 181500., 223500., 140000., 250000.]),
 tensor([[185903.6719],
         [180518.8906],
         [179615.2031],
         [176826.2812],
         [179946.4531]]))